In [1]:
import cv2
import numpy as np
from my_function import imshow

## 图像轮廓检测

轮廓检测的基本概念：

- 轮廓: 图图像中具有相同颜色或强度的连续点的曲线
- 轮廓层次结构: 轮廓之间的嵌套关系，例如一个轮廓是否包含另一个轮廓
- 轮廓特征: 轮廓的面积、周长、边界矩形、最小外接矩形、最小外接圆等

常用函数：

|函数名称|功能描述|
|-----|-----|
|cv2.findContours()|查找图像中的轮廓|
|cv2.drawContours()|在图像上绘制轮廓|
|cv2.contourArea()|计算轮廓的面积|
|cv2.arcLength()|计算轮廓的周长或弧长|
|cv2.boundingRect()|计算轮廓的边界矩形|
|cv2.minAreaRect()|计算轮廓的最小外接矩形|
|cv2.minEnclosingCircle()|计算轮廓的最小外接圆|
|cv2.approxPolyDP()|对轮廓进行多边形近似|


## 查找并绘制轮廓

函数原型：

`contours, hierarchy = cv2.findContours(image, mode, method[, contours[, hierarchy[, offset]]])`

参数说明：

- image: 输入的二值图像（通常为经过阈值处理或边缘检测后的图像）
- mode: 轮廓检索模式，常用的有：
  - `cv2.RETR_EXTERNAL`: 只检测最外层轮廓
  - `cv2.RETR_LIST`: 检测所有轮廓，但不建立层次关系
  - `cv2.RETR_TREE`: 检测所有轮廓，并建立完整的层次结构
- method: 轮廓近似方法，常用的有：
  - `cv2.CHAIN_APPROX_NONE`: 存储所有的轮廓点
  - `cv2.CHAIN_APPROX_SIMPLE`: 压缩水平、垂直和对角线段，只保留端点
- contours: 输出的轮廓列表，每个轮廓是一个点集
- hierarchy: 输出的层次结构信息
- offset: 可选参数，轮廓点的偏移量

返回值：

- contours: 轮廓列表
- hierarchy: 轮廓的层次结构信息

函数原型：

`cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])`


参数说明：

- image: 要绘制轮廓的图像
- contours: 轮廓列表
- contourIdx: 要绘制的轮廓索引，如果为负数，则绘制所有轮廓
- color: 轮廓的颜色
- thickness: 轮廓线的厚度，如果为负数，则填充轮廓内部
- lineType: 线型
- hierarchy: 轮廓的层次结构信息
- maxLevel: 绘制的最大层次深度
- offset: 轮廓点的偏移量

In [2]:
img=cv2.imread("./images/opencv_logo.jpg")
_,binary=cv2.threshold(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

contours, hierarchy=cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
img1=img.copy()
cv2.drawContours(img1,contours,-1,(0,0,0),3)
imshow(img1=img1)

## 计算轮廓边长与面积

- 计算轮廓面积：

  `area = cv2.contourArea(contour[, oriented])`

  参数说明：
    - contour: 输入的轮廓
    - oriented: 可选参数，是否计算有向面积

- 计算轮廓周长或弧长：

  `perimeter = cv2.arcLength(curve, closed)`

  参数说明：
    - curve: 输入的轮廓
    - closed: 布尔值，表示轮廓是否闭合


## 计算轮廓边界矩形

计算轮廓的边界矩形：

`x, y, w, h = cv2.boundingRect(contour)`

参数说明：
- contour: 输入的轮廓
- 返回值：矩形的左上角坐标 (x, y) 以及宽度 w 和高度 h

In [26]:
img2=img.copy()
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)

imshow(img2=img2)

## 计算轮廓最小外接矩形/最小外接圆

- 计算轮廓的最小外接矩形：

  `rect = cv2.minAreaRect(contour)`

  参数说明：

  - contour: 输入的轮廓
  - 返回值：包含矩形中心坐标、宽高和旋转角度的元组 

- 计算轮廓的最小外接圆：

  `center, radius = cv2.minEnclosingCircle(contour)`

  参数说明：
  
  - contour: 输入的轮廓
  - 返回值：圆心坐标和半径


In [27]:
img3=img.copy()
for contour in contours:
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = box.astype(int) # 转换为整数坐标
    cv2.drawContours(img3, [box], 0, (0, 0, 255), 2)

imshow(img3=img3)

In [28]:
img4=img.copy()
for contour in contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    radius = int(radius)
    cv2.circle(img4, center, radius, (255, 0, 0), 2)

imshow(img4=img4)

## 多边形近似轮廓

对轮廓进行多边形近似：

`approx = cv2.approxPolyDP(curve, epsilon, closed)`

参数说明：

- curve: 输入的轮廓
- epsilon: 近似精度，表示原始轮廓与近似轮廓之间的最大距离
- closed: 布尔值，表示轮廓是否闭合

返回值：

- approx: 近似后的轮廓点集

In [22]:
img5=img.copy()
# 用于保存所有角点的列表
all_corners = []
for contour in contours:
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    cv2.drawContours(img5, [approx], 0, (0, 255, 0), 2)
    # 将当前轮廓的角点添加到所有角点列表中
    all_corners.extend(approx.reshape(-1, 2))

img6=img.copy()
# 绘制所有角点
for point in all_corners:
    x, y = point
    cv2.circle(img6, (x, y), 5, (0, 0, 255), -1)

imshow(img5=img5, img6=img6)

## 轮廓检测实战

检测一本书的外部轮廓

In [3]:
# 读取图像并二值化
book=cv2.imread("./images/book.jpg")
gray_book=cv2.cvtColor(book,cv2.COLOR_BGR2GRAY)
_,binary_book=cv2.threshold(gray_book,166,255,cv2.THRESH_BINARY) # 阈值需要反复调整
# 使用形态学梯度检出边缘
edge_book=cv2.morphologyEx(binary_book,cv2.MORPH_GRADIENT,np.ones((5,5),np.uint8))
# 寻找轮廓
contours,_=cv2.findContours(edge_book, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 筛选书本轮廓
book_contour = None
max_area = 0
for contour in contours:
    # 计算轮廓面积
    area = cv2.contourArea(contour)
    # 过滤掉太小的轮廓
    if area < 1000:
        continue
    # 多边形近似
    epsilon=0.02*cv2.arcLength(contour, True)
    approx=cv2.approxPolyDP(contour, epsilon, True)
    # 寻找四边形（书本通常是四边形）
    if len(approx) == 4:
        # 选择面积最大的四边形
        if area > max_area:
            max_area = area
            book_contour = approx
# 输出结果
if book_contour is None:
    print("未找到书本轮廓")
else:
    print("找到书本轮廓")
    cv2.drawContours(book, [book_contour], -1, (0, 0, 255), 3)
    imshow(book_with_contour=book)

找到书本轮廓
